In [1]:
# Slighlty updated for Keras 2.0 APIs
# - by Kyeong Soo (Joseph) Kim

import datetime
import os
import numpy as np
import pandas as pd
os.environ['TF_CPP_MIN_LOG_LEVEL']='2' # supress warning messages
import tensorflow as tf

from keras.layers import Dense, Dropout
from keras.models import Sequential
from sklearn.preprocessing import scale
from timeit import default_timer as timer

Using TensorFlow backend.


In [2]:
path_train = "../data/UJIIndoorLoc/trainingData2.csv"
path_validation = "../data/UJIIndoorLoc/validationData2.csv"
path_results = "../results/indoor_localization-2" # TODO: implement auto-detection of the file name

In [3]:
#Explicitly pass header=0 to be able to replace existing names 
train_df = pd.read_csv(path_train,header = 0)
train_df = train_df[:19930]
train_AP_strengths = train_df.iloc[:,:520] #select first 520 columns

#Scale transforms data to center to the mean and component wise scale to unit variance
train_AP_features = scale(np.asarray(train_AP_strengths).astype(float), axis=1) # convert integer to float and scale jointly (axis=1)

#The following two objects are actually pandas.core.series.Series objects
building_ids_str = train_df["BUILDINGID"].map(str) #convert all the building ids to strings
building_floors_str = train_df["FLOOR"].map(str) #convert all the building floors to strings

res = building_ids_str + building_floors_str #element wise concatenation of BUILDINGID+FLOOR
train_labels = np.asarray(building_ids_str + building_floors_str)

#convert labels to categorical variables, dummy_labels has type 'pandas.core.frame.DataFrame'
dummy_labels = pd.get_dummies(train_labels)


"""one hot encode the dummy_labels.
this is done because dummy_labels is a dataframe with the labels (BUILDINGID+FLOOR) 
as the column names
"""
train_labels = np.asarray(dummy_labels) #labels is an array of shape 19937 x 13. (there are 13 types of labels)

In [4]:
#generate len(train_AP_features) of floats in between 0 and 1
train_val_split = np.random.rand(len(train_AP_features))
#convert train_val_split to an array of booleans: if elem < 0.7 = true, else: false
train_val_split = train_val_split < 0.70 #should contain ~70% percent true

We aren't given a formal testing set, so we will treat the given validation set as the testing set

In [5]:
# We will then split our given training set into training + validation 
train_X = train_AP_features[train_val_split]
train_y = train_labels[train_val_split]
val_X = train_AP_features[~train_val_split]
val_y = train_labels[~train_val_split]

In [6]:
#Turn the given validation set into a testing set
test_df = pd.read_csv(path_validation,header = 0)
test_AP_features = scale(np.asarray(test_df.iloc[:,0:520]).astype(float))
test_labels = np.asarray(test_df["BUILDINGID"].map(str) + test_df["FLOOR"].map(str))
test_labels = np.asarray(pd.get_dummies(test_labels))

In [7]:
epochs = 20
batch_size = 10
input_size = 520
num_classes = 13
nums_hidden_nodes = [16] # for test
# nums_hidden_nodes = [16, 32, 64, 128, 256, 512]
dropout_rates = [0.5] # for test
# dropout_rates = [0.0, 0.2, 0.5]
# dropout_rates = np.arange(11)*0.05 # 0.0, 0.05,...,0.5
losses = np.zeros((len(nums_hidden_nodes), len(dropout_rates)))
accuracies = np.zeros_like(losses)

In [8]:
def classifier(num_hidden_nodes, dropout_rate):
    model = Sequential()
    model.add(Dense(num_hidden_nodes, input_dim=input_size, activation='relu', use_bias=True))
#     model.add(Dropout(dropout_rate))
#     model.add(Dense(256, activation='relu', use_bias=True))
#     model.add(Dropout(dropout_rate))
#     model.add(Dense(128, activation='relu', use_bias=True))
#     model.add(Dropout(dropout_rate))
#     model.add(Dense(64, activation='relu', use_bias=True))
#     model.add(Dropout(dropout_rate))
    model.add(Dense(num_classes, activation='softmax', use_bias=True))
    model.compile(optimizer='adam', loss='categorical_crossentropy',metrics=['accuracy'])
    return model

In [9]:
for nh in range(len(nums_hidden_nodes)):
    for dr in range(len(dropout_rates)):
        c = classifier(nums_hidden_nodes[nh], dropout_rates[dr])
        startTime = timer()
        c.fit(train_X, train_y, validation_data=(val_X, val_y), batch_size=batch_size, epochs=epochs)
        elapsedTime = timer() - startTime
        print("Model trained with nh=%d & dr=%.2f in %e s.\n" % (nums_hidden_nodes[nh], dropout_rates[dr], elapsedTime))
        loss, acc = c.evaluate(test_AP_features, test_labels)
        losses[nh, dr] = loss
        accuracies[nh, dr] = acc

Train on 13915 samples, validate on 6015 samples
Epoch 1/20
13915/13915 [==============================] - 2s - loss: 0.2766 - acc: 0.9227 - val_loss: 0.0665 - val_acc: 0.9805
Epoch 2/20
13915/13915 [==============================] - 1s - loss: 0.0371 - acc: 0.9915 - val_loss: 0.0356 - val_acc: 0.9887
Epoch 3/20
13915/13915 [==============================] - 1s - loss: 0.0234 - acc: 0.9944 - val_loss: 0.0361 - val_acc: 0.9880
Epoch 4/20
13915/13915 [==============================] - 1s - loss: 0.0185 - acc: 0.9948 - val_loss: 0.0259 - val_acc: 0.9914
Epoch 5/20
13915/13915 [==============================] - 1s - loss: 0.0141 - acc: 0.9958 - val_loss: 0.0281 - val_acc: 0.9907
Epoch 6/20
13915/13915 [==============================] - 1s - loss: 0.0117 - acc: 0.9966 - val_loss: 0.0268 - val_acc: 0.9900
Epoch 7/20
13915/13915 [==============================] - 1s - loss: 0.0122 - acc: 0.9957 - val_loss: 0.0199 - val_acc: 0.9922
Epoch 8/20
13915/13915 [==============================] - 1s -

In [10]:
now = datetime.datetime.now()
path_results += "_" + now.strftime("%Y%m%d-%H%M%S") + ".csv"
f = open(path_results, 'w')
for nh in range(len(nums_hidden_nodes)):
    for dr in range(len(dropout_rates)):
        f.write("nh=%d & dr=%.2f: loss=%e, accuracy=%e" % (nums_hidden_nodes[nh], dropout_rates[dr], losses[nh, dr], accuracies[nh, dr]))
f.close()